In [11]:
import pandas as pd 
import re
import seaborn as sns 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.preprocessing import StandardScaler


data_train=pd.read_csv('./Data/train.csv')
data_test=pd.read_csv('./Data/test.csv')
ids_test=data_test.Id

data_test=data_test.drop('Id',axis=1)
X=data_train.copy()

#Procesamiento
# Todo a km
X['running']=X['running'].apply(lambda x: float(x.split(' ')[0]) * 1.60934 if x.split(' ')[2]=='miles' else float(x.split(' ')[0]))
data_test['running']=data_test['running'].apply(lambda x: float(x.split(' ')[0]) * 1.60934 if x.split(' ')[2]=='miles' else float(x.split(' ')[0]))

#Sustituir status por la mediana de su categoria
meds={s :X[X['status']==s]['price'].median() for s in X['status'].unique()}

X['status']=X['status'].map(meds)
data_test['status']=data_test['status'].map(meds)

y=X['price']
X=X.drop('price',axis=1)

num_cols=X.select_dtypes(include='number').columns
scaler=StandardScaler()
X[num_cols]=scaler.fit_transform(X[num_cols])
data_test[num_cols]=scaler.transform(data_test[num_cols])

X=pd.get_dummies(X.drop('color',axis=1)).astype(float)
data_test=pd.get_dummies(data_test.drop('color',axis=1)).astype(float)



X, data_test = X.align(data_test, join='left', axis=1)
data_test = pd.get_dummies(data_test)
X, X_tdata_testest = X.align(data_test, join='left', axis=1)




X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, test_size=0.2,random_state=40)

mod=CatBoostRegressor(loss_function='MAE', silent = True,random_state=0,learning_rate= 0.05, l2_leaf_reg= 7, iterations= 200, depth= 6)
mod.fit(X_train_full,y_train)


# Calculate MAE
mae_1 = mean_absolute_error(mod.predict(X_valid_full), y_valid)

#Uncomment to print MAE
print("Mean Absolute Error:" , mae_1)
mae_1 = mean_absolute_error(mod.predict(X_train_full), y_train)
print('Train error:',mae_1)

Mean Absolute Error: 1963.1008954353554
Train error: 1640.1232040061116


In [10]:

preds=mod.predict(data_test)
output = pd.DataFrame({'Id': ids_test,
                        'Predicted_Price': preds})
output.to_csv('submission.csv', index=False)

In [ ]:
#Resultado actual 1835